In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load data

In [ ]:
from pathlib import Path
path = Path(__file__).parent.parent

In [ ]:
# load dữ liệu từ file ratings train và test

train = pd.read_csv(path + "/training_data_new.csv")
test = pd.read_csv(path + "/testing_data.csv")

food_information = pd.read_excel(path + "/Food_description.xlsx")

In [ ]:
food_information.head(3)

,FoodID,Tên món ăn,Vùng miền,Thành phần,"Thể loại (nướng, nước, khô, ...)",mô tả ngắn gọn ( <100 từ)
0,F1,Xôi,"Miền Bắc, Miền Trung, Miền Nam","Gạo nếp, đậu xanh, Vừng, Đậu phộng",Món khô,Xôi được làm từ gạo nếp hấp chín bằng hơi nước...
1,F2,Mì quảng,Miền Trung,"mì,nước lèo, thịt, tôm, cá, trứng , đậu phộng...",Món nước,"Có nguồn gốc ở Quảng Nam, mì quảng là món mì s..."
2,F3,Phở bò,Miền Bắc,"bánh phở, nước lèo, thịt bò, trứng chần, hành ...",Món nước,"Món ăn nổi tiếng, được nhiều người ưu thích. Ă..."


In [ ]:
from scipy.sparse import csr_matrix
df_food = train.pivot(
    index = 'FoodId',
    columns = 'UserID',
    values = 'rating'
).fillna(0)

df_food

UserID,U1,U10,U100,U101,U102,U103,U104,U105,U106,U107,U108,U109,U11,U110,U111,U112,U113,U114,U115,U116,U117,U118,U119,U12,U120,U121,U122,U123,U124,U125,U126,U127,U128,U129,U13,U130,U131,U132,U133,U134,...,U63,U64,U65,U66,U67,U68,U69,U7,U70,U71,U72,U73,U74,U75,U76,U77,U78,U79,U8,U80,U81,U82,U83,U84,U85,U86,U87,U88,U89,U9,U90,U91,U92,U93,U94,U95,U96,U97,U98,U99
FoodId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F1,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,5.0,4.0,0.0,0.0,4.0,2.0,5.0,5.0,0.0,3.0,3.0,3.0,0.0,5.0,0.0,...,0.0,3.0,4.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
F10,0.0,0.0,5.0,0.0,2.0,4.0,5.0,0.0,4.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,5.0,0.0,4.0,5.0,0.0,5.0,4.0,0.0,4.0,4.0,2.0,2.0,4.0,0.0,5.0,3.0,4.0,0.0,5.0,0.0,...,5.0,3.0,5.0,4.0,5.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,2.0,3.0,4.0,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
F11,4.0,4.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,4.0,4.0,4.0,5.0,2.0,5.0,4.0,0.0,5.0,3.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,2.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,2.0,0.0,5.0,3.0,3.0,4.0,0.0,0.0,0.0,5.0,0.0,5.0,3.0,5.0,4.0,0.0,4.0,4.0
F12,0.0,3.0,3.0,0.0,0.0,0.0,4.0,3.0,3.0,0.0,5.0,4.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,3.0,5.0,0.0,5.0,4.0,0.0,2.0,5.0,2.0,5.0,0.0,3.0,0.0,3.0,1.0,3.0,5.0,4.0,...,0.0,0.0,3.0,5.0,5.0,3.0,0.0,5.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,1.0,3.0,5.0,0.0,0.0,4.0,0.0,5.0,0.0,3.0,0.0,4.0,5.0,0.0,4.0,4.0,3.0,4.0,3.0,3.0,0.0,5.0,3.0,1.0
F13,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
F14,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
F15,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,4.0,4.0,4.0,0.0,0.0,4.0,5.0
F16,0.0,4.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,4.0,4.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,4.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,5.0
F17,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


# Một vài hàm lấy dữ liệu

In [ ]:
def get_rating(userid, foodid):
    return (train.loc[((train.UserID== userid)&(train.FoodId== foodid)), 'rating'].iloc[0])
def get_foodid(userid): 
    return (train.loc[(train.UserID==userid), 'FoodId'].tolist())
def get_food_title(foodid):
    return (food_information.loc[(food_information.FoodID==foodid), 'Tên món ăn'].iloc[0])
def get_average_rating_user(userid):
    rating_user = train.loc[((train.UserID== userid)), 'rating']
    return np.mean(rating_user)
def get_average_rating_food(foodid):
    rating_food = train.loc[((train.FoodId== foodid)), 'rating']
    return np.mean(rating_food)

In [ ]:
#Tính rating trung bình cho các món ăn. Nếu user ko nằm trong data thì dùng này để đưa ra khuyến nghị
average_rating_food = {}
for i in train["FoodId"].unique():
    average_rating_food[i]=get_average_rating_food(i)

In [ ]:
average_rating_food

{'F1': 3.5051546391752577,
 'F10': 4.107692307692307,
 'F11': 3.788135593220339,
 'F12': 3.481012658227848,
 'F13': 3.6666666666666665,
 'F14': 3.6511627906976742,
 'F15': 4.048387096774194,
 'F16': 3.3026315789473686,
 'F17': 3.76271186440678,
 'F18': 3.142857142857143,
 'F19': 3.7362637362637363,
 'F2': 3.5166666666666666,
 'F20': 3.9285714285714284,
 'F21': 4.142857142857143,
 'F22': 3.0,
 'F23': 4.08,
 'F24': 3.9134615384615383,
 'F25': 2.8536585365853657,
 'F26': 3.810810810810811,
 'F27': 3.9367088607594938,
 'F28': 3.840909090909091,
 'F29': 3.7285714285714286,
 'F3': 4.105263157894737,
 'F30': 3.1451612903225805,
 'F4': 4.213114754098361,
 'F5': 3.0357142857142856,
 'F6': 3.259259259259259,
 'F7': 3.81,
 'F8': 4.15,
 'F9': 3.7333333333333334}

# Các hàm tính độ tương tự 

## Độ đo Cosine

In [ ]:
def cosine_similarity_user(user1, user2):
    both_watch_count = 0
    for foodid in train.loc[train.UserID == user1, 'FoodId'].to_list():
        for foodid in train.loc[train.UserID == user2, 'FoodId'].tolist():
            both_watch_count +=1
    if both_watch_count == 0:
        return 0

    rating1 = []
    rating2 = []

    for foodid in train.loc[train.UserID == user1, 'FoodId'].to_list():
        if foodid in train.loc[train.UserID == user2, 'FoodId'].tolist():
            rating1.append(get_rating(user1, foodid))
            rating2.append(get_rating(user2, foodid))

    if (norm(rating1)*norm(rating2)) == 0:
        return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

## Độ đo tương tự Peason

In [ ]:
#Cồn thức tính hệ số tương quan pearson dựa vào 2 food
def pearson_correlation_user(user1, user2):
    both_watch_count = []
    for foodid in train.loc[(train.UserID == user1), 'FoodId'].tolist():
        if foodid in train.loc[(train.UserID == user2), 'FoodId'].tolist():
            both_watch_count.append(foodid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([get_rating(user1, foodid) for foodid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([get_rating(user2, foodid) for foodid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(get_rating(user1, foodid)- avg_rating_sum_1)*(get_rating(user2, foodid) - avg_rating_sum_2) for foodid in both_watch_count])
    denominator = sqrt(sum([pow((get_rating(user1, foodid) - avg_rating_sum_1), 2) for foodid in both_watch_count]))*sqrt(sum([pow((get_rating(user2, foodid) - avg_rating_sum_2), 2) for foodid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

# Hàm tính top k người dùng có độ tương đồng cao nhất

In [ ]:
def get_similarity_user(user1, number_of_user, metric_similar):
    user_ids = train.UserID.unique().tolist()

    similarity_score = []
    for userID in user_ids:
        if (userID != user1):
            if (metric_similar == "cosine"):
                similarity_score.append((cosine_similarity_user(user1, userID), userID))
            elif (metric_similar == "pearson"):
                similarity_score.append((pearson_correlation_user(user1, userID), userID))
    similarity_score.sort()
    similarity_score.reverse()
    return similarity_score[:number_of_user]

In [ ]:
get_similarity_user('U237', 10, "cosine")

[(1.0000000000000002, 'U92'),
 (1.0, 'U59'),
 (1.0, 'U240'),
 (1.0, 'U239'),
 (1.0, 'U124'),
 (1.0, 'U104'),
 (0.9999999999999998, 'U227'),
 (0.9984603532054125, 'U77'),
 (0.9962405881956831, 'U66'),
 (0.9962405881956831, 'U128')]

# Hàm khuyến nghị món ăn

## Khuyến nghị dựa trên trung bình đánh giá

In [ ]:
def get_recommendation_tbdg_user(userid, number_of_user, metric_similar):
    user_ids = train.UserID.unique().tolist()
    total = {}
    number_rating = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    number_rating[foodid] = 0
                    total[foodid] += get_rating(user, foodid)
                    if get_rating(user, foodid) != 0:
                        number_rating[foodid] +=1
                else:
                    total[foodid] += get_rating(user, foodid)
                    if get_rating(user, foodid) != 0:
                        number_rating[foodid] +=1
   
    ranking = [(to/number_rating[foodid], foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), foodid, score) for score, foodid in ranking]

    return recomendations

In [ ]:
get_recommendation_tbdg_user("U238", 10, "pearson")

[('Phở bò', 'F3', 5.0),
 ('Bánh xèo tôm', 'F23', 5.0),
 ('Hủ tiếu', 'F20', 5.0),
 ('Bún thịt nướng', 'F15', 4.5),
 ('Phở gà', 'F6', 4.0),
 ('Gỏi gà', 'F28', 4.0),
 ('Gà nướng cơm lam', 'F27', 4.0),
 ('Bò kho', 'F24', 4.0),
 ('Mì quảng', 'F2', 4.0),
 ('Há cảo', 'F29', 3.6666666666666665),
 ('Phá lấu', 'F30', 3.6),
 ('Xôi', 'F1', 3.4285714285714284),
 ('Bánh cuốn', 'F19', 3.3333333333333335),
 ('Bánh bèo', 'F12', 3.25),
 ('Miến gà', 'F5', 3.0),
 ('Bún giò', 'F17', 3.0),
 ('Bánh canh chả cá', 'F13', 3.0),
 ('Cháo lòng', 'F25', 2.3333333333333335),
 ('Cơm tấm', 'F4', 1.0)]

## Khuyến nghị dựa theo trung bình đánh giá có trọng số

In [ ]:
def get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar):
    user_ids = train.UserID.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    total[foodid] += get_rating(user, foodid)*score
                    similarity_sum [foodid] = 0
                    similarity_sum [foodid] +=score
                else:
                    total[foodid] += get_rating(user, foodid)*score
                    similarity_sum [foodid] +=score
                
    ranking = [(to/similarity_sum[foodid], foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), foodid, score) for score, foodid in ranking]

    return recomendations

## Khuyến nghị dựa trên khoảng đánh giá

In [ ]:
def get_recommendation_kdg_user(userid, number_of_user, metric_similar):
    user_ids = train.UserID.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    total[foodid] += (get_rating(user, foodid) - get_average_rating_user(user))*score
                    similarity_sum [foodid] = 0
                    similarity_sum [foodid] +=score
                else:
                    total[foodid] += (get_rating(user, foodid) - get_average_rating_user(user))*score
                    similarity_sum [foodid] +=score
                
    ranking = [((get_average_rating_user(userid) + to/similarity_sum[foodid]), foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), foodid, score) for score, foodid in ranking]

    return recomendations

## Khuyến nghị món ăn

In [ ]:
def get_recommendation_user(userid, number_of_user, metrics_rating, metric_similar):
    if userid not in list(train["UserID"].unique()):
        ranking = [(to, userid) for userid, to in average_rating_food.items()]
        ranking.sort()
        ranking.reverse()
        recomendations = [(get_food_title(foodid), foodid, score) for score, foodid in ranking]
        return recomendations
    if (metrics_rating == "tbdg"):
        recomendations = get_recommendation_tbdg_user(userid, number_of_user, metric_similar)
        return recomendations
    elif (metrics_rating == "tbdgcts"):
        recomendations = get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar)
        return recomendations
    elif (metrics_rating == "kdg"):
        recomendations = get_recommendation_kdg_user(userid, number_of_user, metric_similar)
        return recomendations

In [ ]:
userid = "U10000" # ID của người dùng
number_of_user = 10 # Top K người dùng tương tự
metrics_rating = "kdg" #Phương pháp dự đoán rating , ["tbdg", "tbdgcts", "kdg"]
metric_similar = "pearson" # Phương pháp tính toán độ tương tự, ["cosine", "pearson"]
list_R = get_recommendation_user(userid, number_of_user, metrics_rating, metric_similar)
list_R

[('Cơm tấm', 'F4', 4.213114754098361),
 ('Bún bò', 'F8', 4.15),
 ('Cơm gà xối mỡ', 'F21', 4.142857142857143),
 ('Bánh mì thịt', 'F10', 4.107692307692307),
 ('Phở bò', 'F3', 4.105263157894737),
 ('Bánh xèo tôm', 'F23', 4.08),
 ('Bún thịt nướng', 'F15', 4.048387096774194),
 ('Gà nướng cơm lam', 'F27', 3.9367088607594938),
 ('Hủ tiếu', 'F20', 3.9285714285714284),
 ('Bò kho', 'F24', 3.9134615384615383),
 ('Gỏi gà', 'F28', 3.840909090909091),
 ('Súp cua', 'F26', 3.810810810810811),
 ('Bún đậu mắm tôm', 'F7', 3.81),
 ('Bánh mì ốp la', 'F11', 3.788135593220339),
 ('Bún giò', 'F17', 3.76271186440678),
 ('Bánh cuốn', 'F19', 3.7362637362637363),
 ('Bún riêu', 'F9', 3.7333333333333334),
 ('Há cảo', 'F29', 3.7285714285714286),
 ('Bánh canh chả cá', 'F13', 3.6666666666666665),
 ('Bánh bao', 'F14', 3.6511627906976742),
 ('Mì quảng', 'F2', 3.5166666666666666),
 ('Xôi', 'F1', 3.5051546391752577),
 ('Bánh bèo', 'F12', 3.481012658227848),
 ('Bún mắm', 'F16', 3.3026315789473686),
 ('Phở gà', 'F6', 3.2592

# Đánh giá phương pháp bằng RMSE,MAE trên tập test

In [ ]:
def get_recommendation_4test_user(X_test, number_of_user, metrics_rating, metric_similar):
    X_test_pred = []
    X_test_userid = X_test['UserID'].tolist()
    X_test_FoodId = X_test['FoodId'].tolist()
    print("Dự đoán cho tập test với {} điểm dữ liệu".format(len(X_test_userid)))
    for i in range(len(X_test_userid)):
        list_R = get_recommendation_user(X_test_userid[i], number_of_user, metrics_rating, metrics_similar)
        check = 0
        for j in list_R:
            if(X_test_FoodId[i] == j[1]):
                X_test_pred.append(j[2])
                check = 1
        if(check == 0 ):
            X_test_pred.append(0)
        print('----------dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
    return X_test_pred

## Dự đoán bằng pp TBDGCTS và cosine

In [ ]:
import time
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
start = time.time()
number_similar = 10
metrics_rating = "tbdgcts" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 3.50012201074421
----------dự đoán cho dòng thứ 1 là 3.8
----------dự đoán cho dòng thứ 2 là 4.0
----------dự đoán cho dòng thứ 3 là 3.1666666666666665
----------dự đoán cho dòng thứ 4 là 0
----------dự đoán cho dòng thứ 5 là 5.0
----------dự đoán cho dòng thứ 6 là 3.875
----------dự đoán cho dòng thứ 7 là 3.5
----------dự đoán cho dòng thứ 8 là 2.857142857142857
----------dự đoán cho dòng thứ 9 là 3.0
----------dự đoán cho dòng thứ 10 là 3.0
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 3.0
----------dự đoán cho dòng thứ 13 là 3.0
----------dự đoán cho dòng thứ 14 là 3.0
----------dự đoán cho dòng thứ 15 là 0
----------dự đoán cho dòng thứ 16 là 0
----------dự đoán cho dòng thứ 17 là 3.25
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 4.0
----------dự đoán cho dòng thứ 20 là 2.8
----------dự đoán cho dòng thứ 21 là 3.5555555555555554
----------dự đoá

In [ ]:
X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.2361559892554628
Đánh giá trên độ đo MAE: 0.9933975246643874
Đánh giá trên độ đo precision: 0.2616666666666666
Đánh giá trên độ đo recall: 0.2016983016983017
Đánh giá trên độ đo f1_score: 0.17672727272727273


## Dự đoán và đánh giá bằng pp TBDGCTS và peason

In [ ]:
import time
start = time.time()
number_similar = 10
metrics_rating = "tbdgcts" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 4.5
----------dự đoán cho dòng thứ 1 là 3.8687457713364592
----------dự đoán cho dòng thứ 2 là 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 3 là nan
----------dự đoán cho dòng thứ 4 là 4.0
----------dự đoán cho dòng thứ 5 là 0
----------dự đoán cho dòng thứ 6 là 3.4
----------dự đoán cho dòng thứ 7 là 4.323362132873716
----------dự đoán cho dòng thứ 8 là 3.7142857142857144
----------dự đoán cho dòng thứ 9 là 3.613266702914267
----------dự đoán cho dòng thứ 10 là 3.0
----------dự đoán cho dòng thứ 11 là 3.4730376257919295
----------dự đoán cho dòng thứ 12 là 4.0
----------dự đoán cho dòng thứ 13 là 3.3333333333333335
----------dự đoán cho dòng thứ 14 là 3.0
----------dự đoán cho dòng thứ 15 là 3.5
----------dự đoán cho dòng thứ 16 là 4.0
----------dự đoán cho dòng thứ 17 là 5.0
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 4.0
----------dự đoán cho dòng thứ 20 là 2.75
----------dự đoán cho dòng thứ 21 là 0
----------dự đoán cho dòng thứ 22 là 2.8324002479228927
----------dự đoán cho dòng thứ 23 là 0
----------dự đoán cho dòng thứ 24 là 2.5000000000000004
---------

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.1975045388828496
Đánh giá trên độ đo MAE: 1.0040633238013936
Đánh giá trên độ đo precision: 0.21877691645133504
Đánh giá trên độ đo recall: 0.1567287784679089
Đánh giá trên độ đo f1_score: 0.1374331550802139


## Dự đoán và đánh giá bằng pp TBDG và cosine

In [ ]:
import time
start = time.time()
number_similar = 10
metrics_rating = "tbdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 3.5
----------dự đoán cho dòng thứ 1 là 3.8
----------dự đoán cho dòng thứ 2 là 4.0
----------dự đoán cho dòng thứ 3 là 3.1666666666666665
----------dự đoán cho dòng thứ 4 là 0
----------dự đoán cho dòng thứ 5 là 5.0
----------dự đoán cho dòng thứ 6 là 3.875
----------dự đoán cho dòng thứ 7 là 3.5
----------dự đoán cho dòng thứ 8 là 2.857142857142857
----------dự đoán cho dòng thứ 9 là 3.0
----------dự đoán cho dòng thứ 10 là 3.0
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 3.0
----------dự đoán cho dòng thứ 13 là 3.0
----------dự đoán cho dòng thứ 14 là 3.0
----------dự đoán cho dòng thứ 15 là 0
----------dự đoán cho dòng thứ 16 là 0
----------dự đoán cho dòng thứ 17 là 3.25
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 4.0
----------dự đoán cho dòng thứ 20 là 2.8
----------dự đoán cho dòng thứ 21 là 3.5555555555555554
----------dự đoán cho dòng th

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.2361963663297002
Đánh giá trên độ đo MAE: 0.9934297769740806
Đánh giá trên độ đo precision: 0.2616666666666666
Đánh giá trên độ đo recall: 0.2016983016983017
Đánh giá trên độ đo f1_score: 0.17672727272727273


## Dự đoán và đánh giá bằng pp TBDG và peason


In [ ]:
import time
start = time.time()
number_similar = 10
metrics_rating = "tbdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 4.5
----------dự đoán cho dòng thứ 1 là 3.857142857142857
----------dự đoán cho dòng thứ 2 là 0
----------dự đoán cho dòng thứ 3 là 3.75
----------dự đoán cho dòng thứ 4 là 4.0
----------dự đoán cho dòng thứ 5 là 0
----------dự đoán cho dòng thứ 6 là 3.4
----------dự đoán cho dòng thứ 7 là 4.333333333333333
----------dự đoán cho dòng thứ 8 là 3.7142857142857144
----------dự đoán cho dòng thứ 9 là 3.6
----------dự đoán cho dòng thứ 10 là 3.0
----------dự đoán cho dòng thứ 11 là 3.6666666666666665
----------dự đoán cho dòng thứ 12 là 4.0
----------dự đoán cho dòng thứ 13 là 3.3333333333333335
----------dự đoán cho dòng thứ 14 là 3.0
----------dự đoán cho dòng thứ 15 là 3.5
----------dự đoán cho dòng thứ 16 là 4.0
----------dự đoán cho dòng thứ 17 là 5.0
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 4.0
----------dự đoán cho dòng thứ 20 là 2.75
----------dự đoán cho dòng thứ 21 là 0

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.1799464304345575
Đánh giá trên độ đo MAE: 0.9923974061169184
Đánh giá trên độ đo precision: 0.22285870158210583
Đánh giá trên độ đo recall: 0.16020066889632106
Đánh giá trên độ đo f1_score: 0.1378021978021978


## Dự đoán và đánh giá bằng pp KDG và cosine

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 3.9278767420480816
----------dự đoán cho dòng thứ 1 là 3.5155555555555558
----------dự đoán cho dòng thứ 2 là 3.9345238095238098
----------dự đoán cho dòng thứ 3 là 4.548148148148148
----------dự đoán cho dòng thứ 4 là 0
----------dự đoán cho dòng thứ 5 là 4.9
----------dự đoán cho dòng thứ 6 là 3.922222222222222
----------dự đoán cho dòng thứ 7 là 3.2203703703703703
----------dự đoán cho dòng thứ 8 là 3.3634920634920635
----------dự đoán cho dòng thứ 9 là 4.642857142857142
----------dự đoán cho dòng thứ 10 là 3.3333333333333335
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 3.5
----------dự đoán cho dòng thứ 13 là 3.162301587301587
----------dự đoán cho dòng thứ 14 là 3.767857142857143
----------dự đoán cho dòng thứ 15 là 0
----------dự đoán cho dòng thứ 16 là 0
----------dự đoán cho dòng thứ 17 là 3.486111111111111
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòn

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.077158521001207
Đánh giá trên độ đo MAE: 0.9060026122395772
Đánh giá trên độ đo precision: 0.2
Đánh giá trên độ đo recall: 0.17182817182817184
Đánh giá trên độ đo f1_score: 0.17252979145978153


## Dự đoán và đánh giá bằng pp KDG và peason

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 5.072222222222223
----------dự đoán cho dòng thứ 1 là 3.6397389842726655
----------dự đoán cho dòng thứ 2 là 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 3 là nan
----------dự đoán cho dòng thứ 4 là 4.188888888888888
----------dự đoán cho dòng thứ 5 là 0
----------dự đoán cho dòng thứ 6 là 3.328888888888889
----------dự đoán cho dòng thứ 7 là 4.260753141779884
----------dự đoán cho dòng thứ 8 là 3.9873015873015873
----------dự đoán cho dòng thứ 9 là 4.243852030261946
----------dự đoán cho dòng thứ 10 là 2.938888888888889
----------dự đoán cho dòng thứ 11 là 4.166380318028649
----------dự đoán cho dòng thứ 12 là 3.9074074074074074
----------dự đoán cho dòng thứ 13 là 3.762962962962963
----------dự đoán cho dòng thứ 14 là 2.952777777777778
----------dự đoán cho dòng thứ 15 là 4.511111111111111
----------dự đoán cho dòng thứ 16 là 3.712037037037037
----------dự đoán cho dòng thứ 17 là 5.777777777777778
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 3.355555555555555
----------dự đoán cho dòng thứ 20 là 1.640277777777778
----------dự đoán cho dòng thứ 21 là 0
----------dự đoán cho 

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.0975456044387113
Đánh giá trên độ đo MAE: 0.9236028725977318
Đánh giá trên độ đo precision: 0.20101214574898785
Đánh giá trên độ đo recall: 0.16144755003450653
Đánh giá trên độ đo f1_score: 0.16066180935033395


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Đánh giá bằng MRR và NDCG

## Các hàm dùng chung

In [ ]:
GroundTruth = pd.read_csv(path + "/groundtruth.csv")

In [ ]:
# Đầu vào là 2 danh sách: 
# 1 là danh sách predict của user nào đó có dạng ('Cơm tấm', 'F4', 4.209677419354839),
#                                                ('Cơm gà xối mỡ', 'F21', 4.185185185185185),
#                                                ('Bún bò', 'F8', 4.128205128205129),
# Và tập Ground_truth có sẵn. dạng          UserID	FoodId	rating
#                                       80	U236	F18	    4
#                                       81	U236	F8	    4
#                                       82	U236	F14	    4
# Kết quả trả về là 1 danh sách [0, 1] với ddiều kiện là món ăn khuyến nghị phải có trong ground_truth và rating của ground_truth và predict đều phải >=3
def Get_Predict_List(userid, number_of_user, metrics_rating, metric_similar):
    MRR_predict = []
    predict = get_recommendation_user(userid, number_of_user, metrics_rating, metric_similar)
    if len(predict) > 10:
        top_N =10
    else:
        top_N = len(predict)
    predict = predict[:top_N]
    ground_truth = GroundTruth.loc[GroundTruth["UserID"] == userid].head(top_N)
    for i in range(len(ground_truth)):
        if predict[i][1] in list(ground_truth.FoodId):
            if (((GroundTruth.loc[GroundTruth["FoodId"] == predict[i][1]].rating.iloc[0]) >= 3) and (predict[i][2] >=3)):
                MRR_predict.append(1)
            else:
                MRR_predict.append(0)
        else :
            MRR_predict.append(0)
    return MRR_predict

## Hàm tính MRR

In [ ]:
def RRu_score(MRR_predict):
    RRu = 0
    for i in range(len(MRR_predict)):
        rr = MRR_predict[i]/(i+1)
        RRu+=rr
    return RRu

In [ ]:
RRu_score(X)

0.8416666666666667

In [ ]:
def MRR(ListUser, number_of_user, metrics_rating, metric_similar):
    # ListUser: Danh sách các user GroundTruth
    # number_of_user: Số user tương đồng
    # metrics_rating: Phương pháp đánh giá
    # Metric_similar : "Độ đo cosine"
    # Top N: Số lương Top Danh sách muốn tính cho MRR
    # Kết quả trả về là kết quả MRR cho pp tương ứng
    MRR = 0
    for user in ListUser:
        MRR_predict = Get_Predict_List(user, number_of_user, metrics_rating, metric_similar)
        RRu = RRu_score(MRR_predict)
        MRR += RRu
        print("MRRu score:", RRu)
    return (1/len(ListUser))*MRR

In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]

In [ ]:
MRR_score = MRR(ListUser, 10, "kdg", "cosine")
print(MRR_score)

MRRu score: 1.926190476190476
MRRu score: 1.3194444444444444
MRRu score: 1.925
MRRu score: 2.0194444444444444
MRRu score: 1.8333333333333333
1.8046825396825399


In [ ]:
MRR_score = MRR(ListUser, 10, "kdg", "pearson")
print(MRR_score)

MRRu score: 0.8416666666666667
MRRu score: 0.2
MRRu score: 1.226190476190476
MRRu score: 0.5833333333333333
MRRu score: 1.5
0.8702380952380953


## Hàm tính NDCG

In [ ]:
from sklearn.metrics import ndcg_score
import math

In [ ]:
def DCG_score(predict_ranking):
    DCG = 0
    for i in range(len(predict_ranking)):
        DCG += predict_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def iDCG_score(true_ranking):
    DCG = 0
    for i in range(len(true_ranking)):
        DCG += true_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def NDCG_for_listUser(ListUser, number_of_user, metrics_rating, metric_similar):
    ndcg = 0
    for user in ListUser:
        prec_ranking = Get_Predict_List(user, number_of_user, metrics_rating, metric_similar)
        true_ranking = sorted(prec_ranking, reverse=True)
        print("predict_ranking")
        print(prec_ranking)
        print("true_ranking")
        print(true_ranking)
        dcg = DCG_score(prec_ranking)
        idcg = iDCG_score(true_ranking)
        ndcg += dcg/idcg
        print("ndcg:", dcg/idcg)
    return (1/len(ListUser))*ndcg

In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
ndcg_score = NDCG_for_listUser(ListUser, 10, "kdg", "cosine")

predict_ranking
[1, 0, 1, 1, 1, 0, 1, 0, 0, 0]
true_ranking
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
ndcg: 0.8990671370233444
predict_ranking
[0, 1, 1, 1, 0, 0, 0, 1, 1, 0]
true_ranking
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
ndcg: 0.7387252447042175
predict_ranking
[1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ndcg: 0.8616875151015704
predict_ranking
[1, 0, 1, 1, 1, 0, 0, 1, 1, 0]
true_ranking
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ndcg: 0.8878428156094798
predict_ranking
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
true_ranking
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
ndcg: 1.0


In [ ]:
ndcg_score

0.8774645424877225

In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
ndcg_score = NDCG_for_listUser(ListUser, 10, "kdg", "pearson")

predict_ranking
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
ndcg: 0.6031171482531633
predict_ranking
[0, 0, 0, 0, 1, 0, 0, 0, 0]
true_ranking
[1, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.38685280723454163
predict_ranking
[0, 1, 1, 1, 0, 0, 1, 0, 0, 0]
true_ranking
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
ndcg: 0.7397466333385988
predict_ranking
[0, 0, 1, 1, 0, 0, 0]
true_ranking
[1, 1, 0, 0, 0, 0, 0]
ndcg: 0.5706417189553201
predict_ranking
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
true_ranking
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 1.0


In [ ]:
ndcg_score

0.6600716615563248